# Les optimisations de type "Pricing"

# Introduction

# Sommaire

- 1. Elaborer le meilleur prix d'un objet.
        - Notre problème de base
           - l'équation linéaire de la demande
           - l'équation d'élasticité non linéaire de la demande
        - Modélisation mathématique
        - Résolution avec Python Gekko
- 2. Elaborer le meilleur prix de plusieurs objets.
        - Notre problème de base
        - Modélisation mathématique
        - Résolution